In [ ]:
import numpy as np
import numpy.linalg as la
import cv2
import matplotlib
import matplotlib.pyplot as plt
import sh
from IPython.core.pylabtools import figsize
figsize(40, 20)


%matplotlib inline

In [ ]:
def plot(*images):
    for i, img in enumerate(images):
        plt.subplot(331 + i)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.xticks([]), plt.yticks([])
        
#     mng = plt.get_current_fig_manager()
#     mng.resize(*mng.window.maxsize())

    plt.show()
    
def my_imshow(img):
    cv2.namedWindow(WINDOW_NAME, cv2.WND_PROP_AUTOSIZE)
    cv2.resizeWindow(WINDOW_NAME, 500,500);
    cv2.startWindowThread()
    
    # Display an image
    cv2.imshow(WINDOW_NAME, img)
    cv2.resizeWindow(WINDOW_NAME, 600,600)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape)/2)
    rot_mat = cv2.getRotationMatrix2D(image_center,angle,1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape,flags=cv2.INTER_LINEAR)
    return result



### cv = cv2
WINDOW_NAME = 'Image de Lena'
IMAGE = '/Users/avishayz/workspace/setsolver/corpus/50.png'
img = image = cv2.imread(IMAGE)

In [ ]:
im = img = image = cv2.imread(IMAGE)
approx = 0
def rectify(h):
    h = h.reshape((4,2))
    hnew = np.zeros((4,2),dtype = np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h,axis = 1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]
    
    if (la.norm(hnew[1] - hnew[0]) > la.norm(hnew[2] - hnew[1])):
        hnew = np.roll(hnew, 2)
    return hnew

def getCards(im):
    global approx
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(1,1),1000)
    flag, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
#     my_imshow(im)
    
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    xxx = []
    for card in contours:
        peri = cv2.arcLength(card, True)
        poly_dp = cv2.approxPolyDP(card, 0.02 * peri,True)
        if len(poly_dp) == 4:
            approx = rectify(poly_dp)     
            h = np.array([ [0,0],[285,0],[285,449],[0,449] ],np.float32)

            transform = cv2.getPerspectiveTransform(approx,h)
            warp = cv2.warpPerspective(im, transform, (285,450))
            xxx.append(card)
            yield warp
    cv.drawContours(im, xxx, -1, (0,0,255), 2)
    my_imshow(im)
    
res = list(getCards(im))
plot(*res[:9])